In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

pd.set_option('display.max_columns', 150)

In [87]:
import timeit
import multiprocessing
import threading

In [11]:
df_stats = pd.read_csv('player_stats.csv')
df_stats.drop(df_stats.columns[0], axis = 1, inplace= True)
df_stats_backup = df_stats.copy()

In [12]:
REORDER_COLS = ['player_id', 'player', 'team', 'gm', 'mp', 'fg', 'fga', 'fg_pct', 'fg2', 'fg2a', 'fg2_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

In [13]:
df_stats = df_stats[REORDER_COLS]
df_stats.insert(3, 'dt', df_stats.gm.apply(lambda g: datetime.datetime.strptime(g[:10], "%Y-%m-%d").date() ))
df_stats.sort(['dt', 'gm', 'team', 'player_id'], inplace = True)

df_stats.reset_index(inplace = True, drop = True)

In [25]:
df_stats.insert(4, 'season', df_stats.dt.apply(lambda d:  d.year+1 if d.month > 4 else d.year))

In [14]:
df_stats.insert(6, 'home_team', df_stats.apply(lambda r: r.team in r.gm, axis = 1))

In [35]:
# df_stats_cum = df_stats.copy()
df_stats_backup= df_stats.copy()

In [27]:
for c in df_stats_cum.columns:
    if 'pct' in c:
        df_stats_cum[c] = None

In [ ]:
# numb_cols = df_stats_cum.columns[7:]
# for p in df_stats_cum.player_id.unique()[:20]:
#     print p
#     df_stats_cum.loc[df_stats_cum.player_id == p, numb_cols] = df_stats_cum.loc[df_stats_cum.player_id == p, numb_cols].cumsum(axis = 0)

In [110]:
def break_up_df_by_column(df, column):
    dfs = {}
    columns = df[column].unique()
    for c in columns:
        dfs[c] = df.loc[df[column] == c]
    return dfs

In [111]:
seasons_dfs = break_up_df_by_column(df_stats, 'season')

In [114]:
for s in seasons_dfs:
    print len(seasons_dfs[s])

116727
116658
117311
121732
120999


In [124]:
def break_up_df_by_column(df, column):
    dfs = {}
    columns = df[column].unique()
    for c in columns:
        dfs[c] = df.loc[df[column] == c]
    return dfs

def cumsum_player_stats(df, numb_cols = numb_cols):
    df.loc[:, numb_cols] = df.loc[:, numb_cols].cumsum(axis = 0)
    return df

In [134]:
seasons_dfs = break_up_df_by_column(df_stats, 'season')

df = seasons_dfs[2011]
players = df.player_id.unique()[:50]
df_final = pd.DataFrame()

In [138]:
pool = multiprocessing.Pool(4)

In [135]:
# ~30+ seconds for 50 records
for p in players:
    dftmp = df[df.player_id == p]
    df_final = pd.concat([df_final, cumsum_player_stats(dftmp)])

In [136]:
df_final.head()

,player_id,player,team,dt,season,gm,home_team,mp,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,chris-mcnealy-2,Chris McNealy,california-irvine,2010-11-08,2011,2010-11-08-illinois,False,21,3,5,0.600,1,3,0.333,2,2,1.0,0,0,NaN,0,3,3,1,0,1,2,2,8
4282,chris-mcnealy-2,Chris McNealy,california-irvine,2010-11-13,2011,2010-11-13-southern-california,False,45,5,11,0.933,3,7,0.833,2,4,1.0,0,1,0.0,1,6,7,1,1,1,3,4,12
10267,chris-mcnealy-2,Chris McNealy,california-irvine,2010-11-19,2011,2010-11-19-california-irvine,True,66,9,19,1.433,7,14,1.404,2,5,1.0,0,1,NaN,3,10,13,2,2,1,4,6,20
11699,chris-mcnealy-2,Chris McNealy,california-irvine,2010-11-20,2011,2010-11-20-california-irvine,True,92,14,28,1.989,11,21,1.975,3,7,1.5,0,1,NaN,3,11,14,3,3,1,8,8,31
13395,chris-mcnealy-2,Chris McNealy,california-irvine,2010-11-21,2011,2010-11-21-california-irvine,True,112,16,34,2.322,13,27,2.308,3,7,NaN,1,3,0.5,3,12,15,4,4,1,8,9,36


In [97]:
def get_length_of_dataframe(p, y, numb_cols = numb_cols):
    # print p
    return df_stats.loc[(df_stats.player_id == p) & (df_stats.season == y), numb_cols].cumsum(axis = 0)
    pass

# def test_len(nn = 100):
#     print datetime.datetime.now()
#     df_stats[['player_id', 'season']].drop_duplicates().head(nn).apply(lambda r: get_length_of_dataframe(r.player_id, r.season), axis = 1)
#     print datetime.datetime.now()
    
def test_len(df):
    print datetime.datetime.now()
    df.apply(lambda r: get_length_of_dataframe(r.player_id, r.season), axis = 1)
    print datetime.datetime.now()

In [86]:
test_len(500)

2015-07-23 21:55:43.440090
2015-07-23 21:56:10.743870


In [103]:
df = df_stats[['player_id', 'season']].drop_duplicates().head(500)
res = []

t = threading.Thread(target = test_len, args = df)
res.append(t)
t.start()

# p = multiprocessing.Process(target = test_len, args = df)
# res.append(p)
# p.start()

In [ ]:
# p = multiprocessing.Process(target=apply_find_opp, args=(df,counter,))
# p.start()

In [108]:
p

<Process(Process-6, stopped[1])>

In [83]:
print len(df_stats.player_id.unique())
print len(df_stats[['player_id', 'season']].drop_duplicates())

11913
23371


In [34]:
df_stats_cum.loc[df_stats_cum.player_id.isin(df_stats_cum.player_id.unique()[:20])].sort('player_id').head()

,player_id,player,team,dt,season,gm,home_team,mp,fg,fga,fg_pct,fg2,fg2a,fg2_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts
11,bill-cole-1,Bill Cole,illinois,2010-11-08,2011,2010-11-08-illinois,True,20,2,4,NaN,2,2,NaN,0,2,NaN,0,0,NaN,1,4,5,3,0,1,0,3,4
89,bill-cole-1,Bill Cole,illinois,2010-11-10,2011,2010-11-10-illinois,True,38,4,9,NaN,3,3,NaN,1,6,NaN,0,0,NaN,1,5,6,4,2,3,0,5,9
3745,bill-cole-1,Bill Cole,illinois,2010-11-13,2011,2010-11-13-illinois,True,55,5,12,NaN,4,5,NaN,1,7,NaN,0,0,NaN,3,7,10,4,3,3,1,6,11
10085,bill-cole-1,Bill Cole,illinois,2010-11-18,2011,2010-11-18-texas,False,72,7,17,NaN,4,6,NaN,3,11,NaN,3,4,NaN,3,7,10,4,3,5,1,8,20
10585,bill-cole-1,Bill Cole,illinois,2010-11-19,2011,2010-11-19-illinois,True,94,8,22,NaN,4,7,NaN,4,15,NaN,3,6,NaN,4,8,12,6,4,5,1,8,23
